# XAI Experiment Data Cleaning

In [1]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# from pandas.io.json import json_normalize

In [2]:
# load JSON
def load_data(filename):
    df = pd.read_json(filename)
    data_sorted = df['formAnswers'].apply(pd.Series).join(df[['swymerId', 'workDurationInSeconds']])
    return data_sorted.fillna("")

In [3]:
# from google.colab import files

# uploaded = files.upload()

In [3]:
#load json files
v1_q = load_data('data/answers-XAI2-v1-basic-group-2-qualification.json')
v1_m = load_data('data/answers-XAI2-v1-basic-group-2-main.json')

v2_q = load_data('data/answers-XAI2-v2-salient-group-2-qualification.json')
v2_m = load_data('data/answers-XAI2-v2-salient-group-2-main.json')

# v2_q_p2 = load_data('data/answers-XAI2-v2-salient-group-2-qualification-part2.json')
# v2_m_p2 = load_data('data/answers-v2-salient-group-2-main-part2.json')

# v2_q = pd.concat([v2_q_p1, v2_q_p2])
# v2_m = pd.concat([v2_m_p1, v2_m_p2])

v3_q = load_data('data/answers-XAI2-v3-explanations-group-2-qualification.json')
v3_m = load_data('data/answers-XAI2-v3-explanations-group-2-main.json')

In [4]:
#confert to pandas data frame
df_v1_q = pd.DataFrame(v1_q)
df_v1_m = pd.DataFrame(v1_m)

df_v2_q = pd.DataFrame(v2_q)
df_v2_m = pd.DataFrame(v2_m)

df_v3_q = pd.DataFrame(v3_q)
df_v3_m = pd.DataFrame(v3_m)


In [8]:
# remove columns from df_XX_q which are redundant or not needed for the analysis
reduntant_columns = [
  "METADATA.FEATURE",
  "METADATA.GROUP",
  "confirm.0",
]

# df_v1_q = df_v1_q.drop(columns=reduntant_columns)
# df_v2_q = df_v2_q.drop(columns=reduntant_columns)
# df_v3_q = df_v3_q.drop(columns=reduntant_columns)

# get a list of all column names starting with "tutorial."
tutorial_columns = [col for col in df_v1_q.columns if col.startswith("tutorial.")]
tutorial_columns.append("understand-task")

# remove tutorial columns from both dataframes -> Only qualif
df_v1_q = df_v1_q.drop(columns=tutorial_columns)
# df_v1_m = df_v1_m.drop(columns=tutorial_columns)

df_v2_q = df_v2_q.drop(columns=tutorial_columns)
# df_v2_m = df_v2_m.drop(columns=tutorial_columns)

df_v3_q = df_v3_q.drop(columns=tutorial_columns)
# df_v3_m = df_v3_m.drop(columns=tutorial_columns)

# remove "METADATA.PART" columns from both dataframes
df_v1_q = df_v1_q.drop(columns=["METADATA.PART"])
df_v1_m = df_v1_m.drop(columns=["METADATA.PART"])

df_v2_q = df_v2_q.drop(columns=["METADATA.PART"])
df_v2_m = df_v2_m.drop(columns=["METADATA.PART"])

df_v3_q = df_v3_q.drop(columns=["METADATA.PART"])
df_v3_m = df_v3_m.drop(columns=["METADATA.PART"])

KeyError: "['understand-task'] not found in axis"

In [9]:
# merge df_v1_q and df_v1_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v1 = pd.merge(df_v1_q, df_v1_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v1

,personal-code,newsitem.7.rating-before-xai,newsitem.7.rating-after-xai,POINTS.qualification,newsitem.7.control-question,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,19713,-1,-1,0,wrong-1,,,,,,...,5,3,5,wrong-1,5,3,-1,-1,2,178
1,svm73,73,62,0,wrong-1,,,,,,...,4,4,5,wrong-1,4,4,69,65,4,432
2,VJT09,65,84,0,wrong-2,,,,,,...,6,6,6,correct,5,7,74,-1,6,1178
3,Abc12,80,85,2,correct,7,-1,7,7,7,...,7,7,7,correct,7,5,57,-1,7,1245
4,SKI877,56,49,2,correct,6,-1,6,6,6,...,4,3,4,correct,5,5,41,-1,4,1127
5,VJA94,48,77,2,correct,5,80,6,6,7,...,4,4,7,correct,4,5,74,-1,6,1219
6,AHL41,61,82,2,correct,7,-1,7,7,7,...,6,6,6,correct,6,6,85,-1,6,702
7,CGZ63,73,83,2,correct,7,-1,7,7,7,...,7,7,7,correct,7,7,47,-1,7,1835
8,Bah45,72,86,2,correct,6,-1,6,6,7,...,6,6,6,correct,6,5,83,-1,6,906
9,XXX99,45,63,2,correct,5,-1,5,6,3,...,1,3,6,correct,7,5,69,-1,4,1457


In [10]:
# dataframe of participants who did the qualification task for v1 and the main task for v2
pd.merge(df_v1_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))

,personal-code,newsitem.7.rating-before-xai,newsitem.7.rating-after-xai,POINTS.qualification,newsitem.7.control-question,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,X,-1,-1,0,wrong-1,6,-1,6,6,7,...,7,6,7,correct,5,6,-1,-1,5,800
1,msf07,83,91,1,correct,7,-1,7,7,7,...,7,5,5,correct,7,6,82,-1,5,788
2,DJI91,79,79,2,correct,7,-1,7,7,7,...,5,5,5,correct,5,6,64,-1,5,722
3,MWP95,55,80,2,correct,5,-1,5,6,5,...,3,4,6,correct,3,3,72,-1,3,700
4,Vrg45,60,69,2,correct,6,-1,6,7,7,...,6,6,5,correct,4,6,50,-1,6,1751
5,GD12,78,77,1,correct,7,-1,7,7,7,...,6,5,7,correct,7,6,84,-1,5,1607
6,CRO51,85,85,1,correct,6,-1,7,6,7,...,5,6,6,correct,4,5,91,-1,5,1252
7,HYL08,85,85,2,correct,6,-1,7,6,7,...,6,6,6,correct,5,6,55,-1,6,1000
8,LJN68,58,86,0,wrong-1,7,-1,7,7,7,...,5,4,4,correct,1,3,58,-1,4,841
9,NAK98,-1,-1,2,correct,4,-1,4,4,4,...,5,4,6,correct,6,3,79,46,4,1144


In [11]:
# dataframe of participants who are in the main task of v2, but not in either qualification task
df_v2_m[~df_v2_m.swymerId.isin(df_v1_q.swymerId) & ~df_v2_m.swymerId.isin(df_v2_q.swymerId)]

,information-literacy.identify-sdg,newsitem.4.system-evaluation.indications-useful-concreteness,newsitem.6.system-evaluation.indications-useful,news-dashboard-evaluation.assist-decisions,newsitem.1.control-question,ai-system-evaluation.most-useful-explanation-feature,newsitem.5.system-evaluation.xai-features-useful,newsitem.1.system-evaluation.xai-features-useful,newsitem.2.rating-before-xai,ai-system-evaluation.easy-to-follow-what-system-does,...,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,swymerId,workDurationInSeconds


In [12]:
# merge df_v2_q and df_v2_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v2 = pd.merge(df_v2_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v2

,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,personal-code,newsitem.7.rating-before-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,newsitem.8.system-evaluation.classified-correctly,newsitem.7.system-evaluation.xai-features-useful,newsitem.8.system-evaluation.xai-features-useful,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,6,-1,SJS97,67,6,6,7,7,6,4,...,3,4,5,correct,4,3,58,-1,4,796
1,7,-1,FDA73,66,7,7,7,3,7,2,...,7,7,7,correct,7,7,58,-1,7,3610
2,7,-1,jrc88,86,7,7,7,7,7,7,...,7,7,6,correct,7,7,86,15,7,691
3,7,-1,JDP35,54,7,7,7,7,7,7,...,6,6,4,wrong-2,2,6,51,-1,6,590
4,6,-1,DLL27,83,6,6,6,2,6,2,...,6,5,5,correct,4,6,75,-1,5,980
5,7,-1,BJI91,68,7,7,6,7,7,7,...,5,5,5,correct,5,6,64,-1,5,722
6,5,-1,WEM51,80,7,5,6,7,5,5,...,6,2,7,correct,7,4,72,-1,3,1795
7,7,-1,maf07,72,7,7,7,7,7,7,...,7,5,5,correct,7,6,82,-1,5,788
8,7,-1,CBL01,81,7,7,7,7,7,7,...,7,6,6,correct,6,5,55,-1,7,755
9,6,-1,X,-1,6,7,7,7,6,7,...,7,6,7,correct,5,6,-1,-1,5,800


In [13]:
df_v3 = pd.merge(df_v3_q, df_v3_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v3

,newsitem.8.rating-after-xai,personal-code,newsitem.8.control-question,newsitem.8.rating-before-xai,POINTS.qualification,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.7.rating-before-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,23,YHS49,correct,-1,2,5,76,5,4,3,...,5,6,7,correct,5,5,56,60,4,1005
1,-1,Pzf94,correct,-1,2,6,67,7,7,7,...,6,5,6,correct,4,5,62,-1,5,2277
2,55,X,correct,-1,2,5,87,5,6,6,...,6,4,4,wrong-2,3,4,75,53,7,327
3,-1,X,wrong-1,-1,0,,,,,,...,6,7,6,correct,7,6,-1,-1,6,1273
4,23,1955,correct,-1,2,7,70,7,4,6,...,7,6,7,correct,6,6,56,15,6,1142
5,-1,Sib61,wrong-1,56,0,,,,,,...,6,5,7,correct,7,6,55,57,4,354
6,-1,SDW29,correct,-1,2,7,64,7,7,5,...,5,3,7,correct,6,5,93,-1,6,1757
7,-1,ML70,correct,-1,2,6,100,6,4,2,...,4,4,6,correct,5,6,100,-1,4,1646
8,-1,IPH97,correct,-1,2,7,61,7,7,7,...,5,5,6,correct,7,6,59,-1,5,849
9,-1,SIC41,correct,100,2,7,-1,7,7,2,...,7,6,6,correct,6,6,66,-1,2,2376


In [14]:
df = pd.concat([df_v1, df_v2, df_v3], axis=0, ignore_index=True)
# df["is-journalist"] = False
df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
df["workDurationInSeconds"] = df["workDurationInSeconds.main"] + df["workDurationInSeconds.qualification"]

/tmp/ipykernel_75075/2953061995.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
/tmp/ipykernel_75075/2953061995.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["workDurationInSeconds"] = df["workDurationInSeconds.main"] + df["workDurationInSeconds.qualification"]


In [15]:
columns_to_rename = {
  "swymerId": "METADATA.swymerId",
  "workDurationInSeconds.main": "METADATA.workDurationInSeconds.main",
  "workDurationInSeconds.qualification": "METADATA.workDurationInSeconds.qualification",
  "workDurationInSeconds": "METADATA.workDurationInSeconds",
  "personal-code": "METADATA.personal-code",
  # "is-journalist": "JOURNALIST",
  "METADATA.FEATURE": "FEATURE",
}

df = df.rename(columns=columns_to_rename)

In [16]:
# order columns alphabetically
df = df.reindex(sorted(df.columns), axis=1)

In [17]:
from pprint import pprint

columns = list(df.columns)
pprint(columns)

['FEATURE',
 'METADATA.GROUP',
 'METADATA.personal-code',
 'METADATA.swymerId',
 'METADATA.workDurationInSeconds',
 'METADATA.workDurationInSeconds.main',
 'METADATA.workDurationInSeconds.qualification',
 'POINTS',
 'POINTS.main',
 'POINTS.qualification',
 'ai-system-evaluation.additional-functionality',
 'ai-system-evaluation.benefit-of-doubt',
 'ai-system-evaluation.classification-comprehensible',
 'ai-system-evaluation.easy-to-follow-what-system-does',
 'ai-system-evaluation.know-what-will-happen-next-time',
 'ai-system-evaluation.most-useful-explanation-feature',
 'ai-system-evaluation.other-information-wish',
 'ai-system-evaluation.rely-on-ai-system',
 'ai-system-evaluation.sceptical-about-ai-system',
 'ai-system-evaluation.system-able-to-classify-news-articles',
 'ai-system-evaluation.system-able-to-detect-SDG',
 'ai-system-evaluation.system-can-correctly-classify-news-articles',
 'ai-system-evaluation.system-can-efficiently-classify-news-articles',
 'ai-system-evaluation.system-

In [18]:
# determine which columns only contain values which can be parsed to numbers
def is_number(s):
    if s == "":
        return True # empty cells are treated as possible numbers
    try:
        float(s)
        return True
    except ValueError:
        return False

In [19]:
# determine which columns only contain values which can be parsed to integers or are empty
# change data type of numeric columns to int
numeric_columns = df.columns[df.applymap(is_number).all()]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, downcast='integer')

/tmp/ipykernel_75075/1070754137.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  numeric_columns = df.columns[df.applymap(is_number).all()]


In [20]:
rating_columns = [col for col in df.columns if ".rating-" in col]

In [21]:
df

,FEATURE,METADATA.GROUP,METADATA.personal-code,METADATA.swymerId,METADATA.workDurationInSeconds,METADATA.workDurationInSeconds.main,METADATA.workDurationInSeconds.qualification,POINTS,POINTS.main,POINTS.qualification,...,newsitem.7.system-evaluation.xai-features-useful,newsitem.8.control-question,newsitem.8.rating-after-xai,newsitem.8.rating-before-xai,newsitem.8.system-evaluation.classified-correctly,newsitem.8.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.system-evaluation.indications-useful,newsitem.8.system-evaluation.indications-useful-concreteness,newsitem.8.system-evaluation.understand-what-system-does,newsitem.8.system-evaluation.xai-features-useful
0,basic,2,19713,CX_79169039921,258,178,80,3,3,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,basic,2,svm73,CX_31975747433,1721,432,1289,2,2,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,basic,2,VJT09,CX_381218321691,1752,1178,574,5,5,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,basic,2,Abc12,CX_971252466090,5667,1245,4422,7,5,2,...,7.0,correct,-1.0,-1.0,7.0,7.0,7.0,7.0,7.0,7.0
4,basic,2,SKI877,CX_42129048083,1920,1127,793,7,5,2,...,6.0,correct,-1.0,-1.0,6.0,6.0,6.0,6.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,explanations,2,AJH75,CX_781311083642,1471,1048,423,7,5,2,...,6.0,correct,-1.0,-1.0,6.0,6.0,6.0,6.0,6.0,5.0
156,explanations,2,TWH57,CX_731451060442,1461,709,752,7,5,2,...,7.0,correct,-1.0,-1.0,7.0,6.0,5.0,5.0,7.0,6.0
157,explanations,2,STT55,CX_211435315665,3505,1720,1785,7,5,2,...,6.0,correct,-1.0,40.0,7.0,6.0,7.0,6.0,7.0,7.0
158,explanations,2,Mka73,CX_701215841281,4101,3098,1003,6,4,2,...,7.0,correct,-1.0,-1.0,7.0,7.0,7.0,7.0,7.0,7.0


In [22]:
# check all value in rating columns which are >= -1 
print((df[rating_columns] < -1).eq(False).all())


newsitem.1.rating-after-xai     True
newsitem.1.rating-before-xai    True
newsitem.2.rating-after-xai     True
newsitem.2.rating-before-xai    True
newsitem.3.rating-after-xai     True
newsitem.3.rating-before-xai    True
newsitem.4.rating-after-xai     True
newsitem.4.rating-before-xai    True
newsitem.5.rating-after-xai     True
newsitem.5.rating-before-xai    True
newsitem.6.rating-after-xai     True
newsitem.6.rating-before-xai    True
newsitem.7.rating-after-xai     True
newsitem.7.rating-before-xai    True
newsitem.8.rating-after-xai     True
newsitem.8.rating-before-xai    True
dtype: bool


In [23]:

# check all value in rating columns which are <= 100 
print((df[rating_columns] > 100).eq(False).all())

newsitem.1.rating-after-xai     True
newsitem.1.rating-before-xai    True
newsitem.2.rating-after-xai     True
newsitem.2.rating-before-xai    True
newsitem.3.rating-after-xai     True
newsitem.3.rating-before-xai    True
newsitem.4.rating-after-xai     True
newsitem.4.rating-before-xai    True
newsitem.5.rating-after-xai     True
newsitem.5.rating-before-xai    True
newsitem.6.rating-after-xai     True
newsitem.6.rating-before-xai    True
newsitem.7.rating-after-xai     True
newsitem.7.rating-before-xai    True
newsitem.8.rating-after-xai     True
newsitem.8.rating-before-xai    True
dtype: bool


In [24]:
df.to_csv("data/data_cleaned.csv", index=False)